#Step 1: Import Data

1.1 Import packages

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import gc #Garbage Collector to free up memory
gc.enable

<function gc.enable>

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        col_type2 = df[col].dtype.name
        
        if ((col_type != object) and (col_type2 != 'category')):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

1.2 Load Data

In [ ]:
aisles = reduce_mem_usage(pd.read_csv('/content/gdrive/My Drive/Dataset/aisles.csv'))

Memory usage of dataframe is 0.00 MB
Memory usage after optimization is: 0.01 MB
Decreased by -201.8%


In [ ]:
departments = reduce_mem_usage(pd.read_csv('/content/gdrive/My Drive/Dataset/departments.csv'))

Memory usage of dataframe is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased by -110.8%


In [ ]:
order_products__prior = reduce_mem_usage(pd.read_csv('/content/gdrive/My Drive/Dataset/order_products__prior.csv'))

Memory usage of dataframe is 989.82 MB
Memory usage after optimization is: 340.25 MB
Decreased by 65.6%


In [ ]:
order_products__train = reduce_mem_usage(pd.read_csv('/content/gdrive/My Drive/Dataset/order_products__train.csv'))

Memory usage of dataframe is 42.26 MB
Memory usage after optimization is: 13.20 MB
Decreased by 68.7%


In [ ]:
orders = reduce_mem_usage(pd.read_csv('/content/gdrive/My Drive/Dataset/orders.csv'))

Memory usage of dataframe is 182.71 MB
Memory usage after optimization is: 45.68 MB
Decreased by 75.0%


In [ ]:
products = reduce_mem_usage(pd.read_csv('/content/gdrive/My Drive/Dataset/products.csv'))

Memory usage of dataframe is 1.52 MB
Memory usage after optimization is: 2.15 MB
Decreased by -41.8%


##1.3 Create DataFrame with the orders and products purchased based on prior orders

In [ ]:
#DataFrame op contains the customers (user_id), orders they have placed (order_id) and products bought in each order (product_id)
orders.set_index('order_id', inplace=True, drop=False)
op = order_products__prior.join(orders, on='order_id', rsuffix='_')
op.drop('order_id_', inplace=True, axis=1)

In [ ]:
op.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


#2. Feature Engineering

Create predictor variables based on user, product and user & product User: describes behavior of a user Product: describes charactersitics of a product User & product: describes the behavior of a user towards a specific product

##2.1 Create user features

In [ ]:
#Create features with the average days between order and total number of orders
#Based on all of the users orders in the order dataset
user_all = pd.DataFrame()
user_all['user_average_days_between_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean()
user_all['user_number_orders'] = orders.groupby('user_id').size()

user_all.head()

,user_average_days_between_orders,user_number_orders
user_id,,
1,19.00000,11
2,16.28125,15
3,12.00000,13
4,17.00000,6
5,11.50000,5


In [ ]:
user_all = user_all.reset_index()

In [ ]:
user_all.head()

,user_id,user_average_days_between_orders,user_number_orders
0,1,19.00000,11
1,2,16.28125,15
2,3,12.00000,13
3,4,17.00000,6
4,5,11.50000,5


In [ ]:
#Create user features based on prior orders
#user_total_orders identifies the highest number of orders for each user based on prior orders
#user_all_products identifies all the products purchased in prior orders
#user_total_distinct_products identifies the number of unique products that were purchsed by a specific user based on their prior orders

user= pd.DataFrame()
user['total_items'] = op.groupby('user_id').size()
user['all_products'] = op.groupby('user_id')['product_id'].apply(set)
user['total_distinct_items'] = (user.all_products.map(len))

In [ ]:
user.head()

,total_items,all_products,total_distinct_items
user_id,,,
1,59,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",18
2,195,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",102
3,88,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",33
4,18,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",17
5,37,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",23


In [ ]:
user = user.reset_index()

In [ ]:
#Join the user_all DataFrame and the user DataFrame
user = pd.merge(user, user_all)

In [ ]:
user.head()

,user_id,total_items,all_products,total_distinct_items,user_average_days_between_orders,user_number_orders
0,1,59,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",18,19.00000,11
1,2,195,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",102,16.28125,15
2,3,88,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",33,12.00000,13
3,4,18,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",17,17.00000,6
4,5,37,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",23,11.50000,5


In [ ]:
del user_all

In [ ]:
#Create feature for the average products in a basket
user['user_avg_basket'] = user.total_items / user.user_number_orders

In [ ]:
user.head()

,user_id,total_items,all_products,total_distinct_items,user_average_days_between_orders,user_number_orders,user_avg_basket
0,1,59,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",18,19.00000,11,5.363636
1,2,195,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",102,16.28125,15,13.000000
2,3,88,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",33,12.00000,13,6.769231
3,4,18,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",17,17.00000,6,3.000000
4,5,37,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",23,11.50000,5,7.400000


In [ ]:
#Create feature which identifies the highest number of orders for each user

user_total_orders = op.groupby('user_id')[['order_number']].max()
user_total_orders.columns = ['user_total_orders']

#To bring in the user_id as a column
user_total_orders = user_total_orders.reset_index()
user_total_orders.head()

,user_id,user_total_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [ ]:
#Join the user_all DataFrame and the user DataFrame
user = pd.merge(user, user_total_orders)

In [ ]:
user.head()

,user_id,total_items,all_products,total_distinct_items,user_average_days_between_orders,user_number_orders,user_avg_basket,user_total_orders
0,1,59,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",18,19.00000,11,5.363636,10
1,2,195,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",102,16.28125,15,13.000000,14
2,3,88,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",33,12.00000,13,6.769231,12
3,4,18,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",17,17.00000,6,3.000000,5
4,5,37,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",23,11.50000,5,7.400000,4


In [ ]:
user = user.drop('all_products', axis=1)

In [ ]:
user.head()

,user_id,total_items,total_distinct_items,user_average_days_between_orders,user_number_orders,user_avg_basket,user_total_orders
0,1,59,18,19.00000,11,5.363636,10
1,2,195,102,16.28125,15,13.000000,14
2,3,88,33,12.00000,13,6.769231,12
3,4,18,17,17.00000,6,3.000000,5
4,5,37,23,11.50000,5,7.400000,4


In [ ]:
#Delete unnecessary DataFrame
del user_total_orders

In [ ]:
#Create feature which shows the ratio of products reordered in the past over the total number of products purchased 
#This give an indiciation of how frequent a customer has reordered products
#Numerator is the number of times a specific user has reordered products 
#Denominator is the number of all the products that have been purchased on all of the user's previous orders

user_reorder = op.groupby('user_id')['reordered'].mean().to_frame('user_reordered_ratio')
user_reorder = user_reorder.reset_index()
user_reorder.head()

,user_id,user_reordered_ratio
0,1,0.694915
1,2,0.476923
2,3,0.625000
3,4,0.055556
4,5,0.378378


In [ ]:
#Merge the user_order DataFrame with the user DataFrame to have all the features based on users

user = pd.merge(user, user_reorder)
del user_reorder

user.head()

,user_id,total_items,total_distinct_items,user_average_days_between_orders,user_number_orders,user_avg_basket,user_total_orders,user_reordered_ratio
0,1,59,18,19.00000,11,5.363636,10,0.694915
1,2,195,102,16.28125,15,13.000000,14,0.476923
2,3,88,33,12.00000,13,6.769231,12,0.625000
3,4,18,17,17.00000,6,3.000000,5,0.055556
4,5,37,23,11.50000,5,7.400000,4,0.378378


2.2 Create product features

In [ ]:
#Create a feature for the number of products purchased
#Do this by creating a specific group for each product and use the size function to count the number of elements

prods = pd.DataFrame()
prods['product_orders_total'] = op.groupby(op.product_id).size()

#Create a feature for the numner of products reordered 

prods['product_reorder_total'] = op['reordered'].groupby(op.product_id).sum()

#create a feature for the rate at which products are reordered
prods['product_reorder_rate'] = prods.product_reorder_total / prods.product_orders_total

In [ ]:
prods.head(n=10)

,product_orders_total,product_reorder_total,product_reorder_rate
product_id,,,
1,1852,1136.0,0.613391
2,90,12.0,0.133333
3,277,203.0,0.732852
4,329,147.0,0.446809
5,15,9.0,0.600000
6,8,3.0,0.375000
7,30,12.0,0.400000
8,165,83.0,0.503030
9,156,82.0,0.525641


In [ ]:
prods = prods.reset_index()
prods.head()

,product_id,product_orders_total,product_reorder_total,product_reorder_rate
0,1,1852,1136.0,0.613391
1,2,90,12.0,0.133333
2,3,277,203.0,0.732852
3,4,329,147.0,0.446809
4,5,15,9.0,0.600000


In [ ]:
#Merge the prods DataFrame with the products dataframe
products = pd.merge(prods,products)


In [ ]:
products.head()

,product_id,product_orders_total,product_reorder_total,product_reorder_rate,product_name,aisle_id,department_id
0,1,1852,1136.0,0.613391,Chocolate Sandwich Cookies,61,19
1,2,90,12.0,0.133333,All-Seasons Salt,104,13
2,3,277,203.0,0.732852,Robust Golden Unsweetened Oolong Tea,94,7
3,4,329,147.0,0.446809,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,15,9.0,0.600000,Green Chile Anytime Sauce,5,13


In [ ]:
products = products.drop('product_name', axis=1)

2.3 Create user-product features

In [ ]:
#Create a feature that indicates how many times a user bought a product
#Do this by creating a specific group that contains all of the rows for each combination of user and product, count the orders, save to a new dataframe UP

up = op.groupby(['user_id', 'product_id'])['order_id'].count().to_frame('up_total_bought')
up.head(n=10)

up_total_bought
user_id product_id                 
1       196                      10
        10258                     9
        10326                     1
        12427                    10
        13032                     3
        13176                     2
        14084                     1
        17122                     1
        25133                     8
        26088                     2

In [ ]:
#Reset the index of the DataFrame to bring in the user_id and product_id 
up = up.reset_index()
up.head()

,user_id,product_id,up_total_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [ ]:
#Create a feature that indicates how often a user bought a product after its first purchase
#Ratio of how many times a user bought a product out of the number of times they could buy it
#Start from first purchase of the product 

#Numerator is the number of times a user purchased the product
number_times = op.groupby(['user_id', 'product_id'])[['order_id']].count()
number_times.columns = ['Number_Times_Purchased']
number_times.head()

Number_Times_Purchased
user_id product_id                        
1       196                             10
        10258                            9
        10326                            1
        12427                           10
        13032                            3

In [ ]:
#Denominator is the total number of orders made since the user first ordered the product
total_orders = op.groupby('user_id')['order_number'].max().to_frame('total_orders')
total_orders = total_orders.reset_index()
total_orders.head()

,user_id,total_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [ ]:
#Identify when the user first purchased the product by creating a distinct group by user_id and product_id, then pull the minimum value from the order_number column
first_order = op.groupby(['user_id', 'product_id'])['order_number'].min().to_frame('first_order')
first_order = first_order.reset_index()
first_order.head()

,user_id,product_id,first_order
0,1,196,1
1,1,10258,2
2,1,10326,5
3,1,12427,1
4,1,13032,2


In [ ]:
#Combine the total_order and first_order dataFrame

range = pd.merge(total_orders, first_order)
range.head()

,user_id,total_orders,product_id,first_order
0,1,10,196,1
1,1,10,10258,2
2,1,10,10326,5
3,1,10,12427,1
4,1,10,13032,2


In [ ]:
#Calculate the number of orders since the user first ordered the product by subtracting range - first order + 1
#The +1 is to account for the first order in which the product was purchased

range['Order_Range_Since_First'] = range.total_orders - range.first_order + 1
range.head()

,user_id,total_orders,product_id,first_order,Order_Range_Since_First
0,1,10,196,1,10
1,1,10,10258,2,9
2,1,10,10326,5,6
3,1,10,12427,1,10
4,1,10,13032,2,9


In [ ]:
#To calculate ratio, first merge the numerator and denominator datasets
up_ratio = pd.merge(number_times, range, on=['user_id', 'product_id'], how='left')
up_ratio.head()


,user_id,product_id,Number_Times_Purchased,total_orders,first_order,Order_Range_Since_First
0,1,196,10,10,1,10
1,1,10258,9,10,2,9
2,1,10326,1,10,5,6
3,1,12427,10,10,1,10
4,1,13032,3,10,2,9


In [ ]:
#Divide numeratior and denominator
up_ratio['up_reorder_ratio'] = up_ratio.Number_Times_Purchased / up_ratio.Order_Range_Since_First
up_ratio.head()

,user_id,product_id,Number_Times_Purchased,total_orders,first_order,Order_Range_Since_First,up_reorder_ratio
0,1,196,10,10,1,10,1.000000
1,1,10258,9,10,2,9,1.000000
2,1,10326,1,10,5,6,0.166667
3,1,12427,10,10,1,10,1.000000
4,1,13032,3,10,2,9,0.333333


In [ ]:
#Drop all other columns besides the final feature, up_reorder_ratio
up_ratio = up_ratio.drop(['Number_Times_Purchased', 'total_orders', 'first_order', 'Order_Range_Since_First'], axis=1)

In [ ]:
up_ratio.head()

,user_id,product_id,up_reorder_ratio
0,1,196,1.000000
1,1,10258,1.000000
2,1,10326,0.166667
3,1,12427,1.000000
4,1,13032,0.333333


In [ ]:
#Remove other DataFrames no longer using
del [number_times, first_order, range]
gc.collect()

238

In [ ]:
#Merge the up_reorder_ratio dataframe with up dataframe which has all of the features based on the user and product interactions
up = pd.merge(up, up_ratio)
up.head()

,user_id,product_id,up_total_bought,up_reorder_ratio
0,1,196,10,1.000000
1,1,10258,9,1.000000
2,1,10326,1,0.166667
3,1,12427,10,1.000000
4,1,13032,3,0.333333


In [ ]:
del up_ratio

In [ ]:
#Create a feature which indicates how many times a customer bought a product on its last 5 orders
#Step 1: Create a new feature ('order_number_reverse) which puts the order_number for each order in reverse (i.e. starts with the order that was purchased the most recently)
#Step 2: Keep only the last five orders for each order
#Step 3: Create a distinct group based on users and products to identify how many times each user bought a product
#Step 4: Create a ratio of the number of times a user bought a product on their last 5 orders / total number of orders (i.e. 5)

In [ ]:
#Step 1: Create the new feature by grouping based on 'user_id', then taking the order_number column and request the highest numner in that column for each group using the .transform.(max) function.
#Subtract the order_number of each row
#Add 1 so that the last order will be marked as first, instead of zero

op['order_number_reverse'] = op.groupby('user_id')['order_number'].transform(max) - op.order_number +1
op.head(n=10)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_number_reverse
0,2,33120,1,1,202279,prior,3,5,9,8.0,6
1,2,28985,2,1,202279,prior,3,5,9,8.0,6
2,2,9327,3,0,202279,prior,3,5,9,8.0,6
3,2,45918,4,1,202279,prior,3,5,9,8.0,6
4,2,30035,5,0,202279,prior,3,5,9,8.0,6
5,2,17794,6,1,202279,prior,3,5,9,8.0,6
6,2,40141,7,1,202279,prior,3,5,9,8.0,6
7,2,1819,8,1,202279,prior,3,5,9,8.0,6
8,2,43668,9,0,202279,prior,3,5,9,8.0,6
9,3,33754,1,1,205970,prior,16,5,17,12.0,10


In [ ]:
#Step 2: Keep only the last five orders for each customer

op_last5 = op[op.order_number_reverse <= 5]
op_last5.head(n=10)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_number_reverse
56,6,40462,1,0,22352,prior,4,1,12,30.0,5
57,6,15873,2,0,22352,prior,4,1,12,30.0,5
58,6,41897,3,0,22352,prior,4,1,12,30.0,5
59,7,34050,1,0,142903,prior,11,2,14,30.0,2
60,7,46802,2,0,142903,prior,11,2,14,30.0,2
97,12,30597,1,1,152610,prior,22,6,8,10.0,5
98,12,15221,2,1,152610,prior,22,6,8,10.0,5
99,12,43772,3,1,152610,prior,22,6,8,10.0,5
100,12,37886,4,1,152610,prior,22,6,8,10.0,5
101,12,37215,5,0,152610,prior,22,6,8,10.0,5


In [ ]:
#Step 3: Create a distinct group based on users and products to identify how many times each user bought a product
#Out of the last 5 orders, create a distinct group based on user_id and product_id that counts how many times a user bought a product

last_five_purchase = op_last5.groupby(['user_id', 'product_id'])[['order_id']].count()
last_five_purchase.columns = ['number_times_last5']
last_five_purchase.head(n=10)

number_times_last5
user_id product_id                    
1       196                          5
        10258                        5
        12427                        5
        13032                        2
        25133                        5
        35951                        1
        38928                        1
        39657                        1
        46149                        3
        49235                        2

In [ ]:
#Merge new feature with up DataFrame which has all of the features for user and product interactions
up = pd.merge(up, last_five_purchase, on=['user_id', 'product_id'], how='left')
del [op_last5, last_five_purchase]
up.head(n=10)

,user_id,product_id,up_total_bought,up_reorder_ratio,number_times_last5
0,1,196,10,1.000000,5.0
1,1,10258,9,1.000000,5.0
2,1,10326,1,0.166667,NaN
3,1,12427,10,1.000000,5.0
4,1,13032,3,0.333333,2.0
5,1,13176,2,0.222222,NaN
6,1,14084,1,0.100000,NaN
7,1,17122,1,0.166667,NaN
8,1,25133,8,1.000000,5.0
9,1,26088,2,0.200000,NaN


In [ ]:
#Instances where number_times_last5 is Nan, this means that the products were not purchased by the customer within the last 5 orders. Thus, we turn the NaN values into zero
up = up.fillna(0)
up.head(n=10)

,user_id,product_id,up_total_bought,up_reorder_ratio,number_times_last5
0,1,196,10,1.000000,5.0
1,1,10258,9,1.000000,5.0
2,1,10326,1,0.166667,0.0
3,1,12427,10,1.000000,5.0
4,1,13032,3,0.333333,2.0
5,1,13176,2,0.222222,0.0
6,1,14084,1,0.100000,0.0
7,1,17122,1,0.166667,0.0
8,1,25133,8,1.000000,5.0
9,1,26088,2,0.200000,0.0


2.4 Merge all features (user, product, userxproduct)

In [ ]:
up.head(n=10)

,user_id,product_id,up_total_bought,up_reorder_ratio,number_times_last5
0,1,196,10,1.000000,5.0
1,1,10258,9,1.000000,5.0
2,1,10326,1,0.166667,0.0
3,1,12427,10,1.000000,5.0
4,1,13032,3,0.333333,2.0
5,1,13176,2,0.222222,0.0
6,1,14084,1,0.100000,0.0
7,1,17122,1,0.166667,0.0
8,1,25133,8,1.000000,5.0
9,1,26088,2,0.200000,0.0


In [ ]:
#Merge up features with user features
#Store the results on a new DataFrame

new_features = up.merge(user, on='user_id', how='left')
new_features.head(n=10)

,user_id,product_id,up_total_bought,up_reorder_ratio,number_times_last5,total_items,total_distinct_items,user_average_days_between_orders,user_number_orders,user_avg_basket,user_total_orders,user_reordered_ratio
0,1,196,10,1.000000,5.0,59,18,19.0,11,5.363636,10,0.694915
1,1,10258,9,1.000000,5.0,59,18,19.0,11,5.363636,10,0.694915
2,1,10326,1,0.166667,0.0,59,18,19.0,11,5.363636,10,0.694915
3,1,12427,10,1.000000,5.0,59,18,19.0,11,5.363636,10,0.694915
4,1,13032,3,0.333333,2.0,59,18,19.0,11,5.363636,10,0.694915
5,1,13176,2,0.222222,0.0,59,18,19.0,11,5.363636,10,0.694915
6,1,14084,1,0.100000,0.0,59,18,19.0,11,5.363636,10,0.694915
7,1,17122,1,0.166667,0.0,59,18,19.0,11,5.363636,10,0.694915
8,1,25133,8,1.000000,5.0,59,18,19.0,11,5.363636,10,0.694915
9,1,26088,2,0.200000,0.0,59,18,19.0,11,5.363636,10,0.694915


In [ ]:
#Merge with products DataFrame
new_features = new_features.merge(products, on='product_id', how='left')
new_features.head(n=10)

,user_id,product_id,up_total_bought,up_reorder_ratio,number_times_last5,total_items,total_distinct_items,user_average_days_between_orders,user_number_orders,user_avg_basket,user_total_orders,user_reordered_ratio,product_orders_total,product_reorder_total,product_reorder_rate,aisle_id,department_id
0,1,196,10,1.000000,5.0,59,18,19.0,11,5.363636,10,0.694915,35791,27791.0,0.776480,77,7
1,1,10258,9,1.000000,5.0,59,18,19.0,11,5.363636,10,0.694915,1946,1389.0,0.713772,117,19
2,1,10326,1,0.166667,0.0,59,18,19.0,11,5.363636,10,0.694915,5526,3603.0,0.652009,24,4
3,1,12427,10,1.000000,5.0,59,18,19.0,11,5.363636,10,0.694915,6476,4797.0,0.740735,23,19
4,1,13032,3,0.333333,2.0,59,18,19.0,11,5.363636,10,0.694915,3751,2465.0,0.657158,121,14
5,1,13176,2,0.222222,0.0,59,18,19.0,11,5.363636,10,0.694915,379450,315913.0,0.832555,24,4
6,1,14084,1,0.100000,0.0,59,18,19.0,11,5.363636,10,0.694915,15935,12923.0,0.810982,91,16
7,1,17122,1,0.166667,0.0,59,18,19.0,11,5.363636,10,0.694915,13880,9377.0,0.675576,24,4
8,1,25133,8,1.000000,5.0,59,18,19.0,11,5.363636,10,0.694915,6196,4586.0,0.740155,21,16
9,1,26088,2,0.200000,0.0,59,18,19.0,11,5.363636,10,0.694915,2523,1360.0,0.539041,23,19


In [ ]:
#Delete DataFrames that we will no longer use
del op, user, products, prods, up
gc.collect()

62

#3.Split into Train and Test DataFrame

In [ ]:
orders_predict = reduce_mem_usage(orders[((orders.eval_set =='train') | (orders.eval_set=='test'))])
orders_predict = orders_predict[['user_id', 'eval_set', 'order_id']]
orders_predict.head(n=10)

Memory usage of dataframe is 4.33 MB
Memory usage after optimization is: 4.33 MB
Decreased by 0.0%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,user_id,eval_set,order_id
order_id,,,
1187899,1,train,1187899
1492625,2,train,1492625
2774568,3,test,2774568
329954,4,test,329954
2196797,5,train,2196797
1528013,6,test,1528013
525192,7,train,525192
880375,8,train,880375
1094988,9,train,1094988


In [ ]:
#Merge the orders_predict DataFrame with the new_features DataFrame 
new_features = reduce_mem_usage(new_features.merge(orders_predict, on='user_id', how='left'))
new_features.head(n=10)

Memory usage of dataframe is 1561.05 MB
Memory usage after optimization is: 647.26 MB
Decreased by 58.5%


,user_id,product_id,up_total_bought,up_reorder_ratio,number_times_last5,total_items,total_distinct_items,user_average_days_between_orders,user_number_orders,user_avg_basket,user_total_orders,user_reordered_ratio,product_orders_total,product_reorder_total,product_reorder_rate,aisle_id,department_id,eval_set,order_id
0,1,196,10,1.000000,5.0,59,18,19.0,11,5.363281,10,0.694824,35791,27791.0,0.776367,77,7,train,1187899
1,1,10258,9,1.000000,5.0,59,18,19.0,11,5.363281,10,0.694824,1946,1389.0,0.713867,117,19,train,1187899
2,1,10326,1,0.166626,0.0,59,18,19.0,11,5.363281,10,0.694824,5526,3603.0,0.651855,24,4,train,1187899
3,1,12427,10,1.000000,5.0,59,18,19.0,11,5.363281,10,0.694824,6476,4797.0,0.740723,23,19,train,1187899
4,1,13032,3,0.333252,2.0,59,18,19.0,11,5.363281,10,0.694824,3751,2465.0,0.657227,121,14,train,1187899
5,1,13176,2,0.222168,0.0,59,18,19.0,11,5.363281,10,0.694824,379450,315913.0,0.832520,24,4,train,1187899
6,1,14084,1,0.099976,0.0,59,18,19.0,11,5.363281,10,0.694824,15935,12923.0,0.811035,91,16,train,1187899
7,1,17122,1,0.166626,0.0,59,18,19.0,11,5.363281,10,0.694824,13880,9377.0,0.675781,24,4,train,1187899
8,1,25133,8,1.000000,5.0,59,18,19.0,11,5.363281,10,0.694824,6196,4586.0,0.740234,21,16,train,1187899
9,1,26088,2,0.199951,0.0,59,18,19.0,11,5.363281,10,0.694824,2523,1360.0,0.539062,23,19,train,1187899


In [ ]:
#Export file into CSV since keep crashing 
new_features.to_csv("new_features.csv",sep = ',')

In [ ]:
new_features.shape

(13307953, 19)

##3.1Create Train DataFrame

In [ ]:
#Filter eval_set for just train to create the train DataFrame
new_features_train = reduce_mem_usage(new_features[new_features.eval_set=='train'])
new_features_train.head(n=10)

Memory usage of dataframe is 412.19 MB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Memory usage after optimization is: 412.19 MB
Decreased by 0.0%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,user_id,product_id,up_total_bought,up_reorder_ratio,number_times_last5,total_items,total_distinct_items,user_average_days_between_orders,user_number_orders,user_avg_basket,user_total_orders,user_reordered_ratio,product_orders_total,product_reorder_total,product_reorder_rate,aisle_id,department_id,eval_set,order_id
0,1,196,10,1.000000,5.0,59,18,19.0,11,5.363281,10,0.694824,35791,27791.0,0.776367,77,7,train,1187899
1,1,10258,9,1.000000,5.0,59,18,19.0,11,5.363281,10,0.694824,1946,1389.0,0.713867,117,19,train,1187899
2,1,10326,1,0.166626,0.0,59,18,19.0,11,5.363281,10,0.694824,5526,3603.0,0.651855,24,4,train,1187899
3,1,12427,10,1.000000,5.0,59,18,19.0,11,5.363281,10,0.694824,6476,4797.0,0.740723,23,19,train,1187899
4,1,13032,3,0.333252,2.0,59,18,19.0,11,5.363281,10,0.694824,3751,2465.0,0.657227,121,14,train,1187899
5,1,13176,2,0.222168,0.0,59,18,19.0,11,5.363281,10,0.694824,379450,315913.0,0.832520,24,4,train,1187899
6,1,14084,1,0.099976,0.0,59,18,19.0,11,5.363281,10,0.694824,15935,12923.0,0.811035,91,16,train,1187899
7,1,17122,1,0.166626,0.0,59,18,19.0,11,5.363281,10,0.694824,13880,9377.0,0.675781,24,4,train,1187899
8,1,25133,8,1.000000,5.0,59,18,19.0,11,5.363281,10,0.694824,6196,4586.0,0.740234,21,16,train,1187899
9,1,26088,2,0.199951,0.0,59,18,19.0,11,5.363281,10,0.694824,2523,1360.0,0.539062,23,19,train,1187899


In [ ]:
#Get the features from the order_products_train DataFrame
new_features_train = reduce_mem_usage(new_features_train.merge(order_products__train[['product_id','order_id','reordered']], on=['product_id', 'order_id'], how='left'))
new_features_train.head(n=10)

In [ ]:
#Fill in 'reordered' column where NaN
new_features_train['reordered'] = new_features_train['reordered'].fillna(0)
new_features_train.head(n=10)

In [ ]:
#Create a unique index that describe each row using user_id and product_id
new_features_train = new_features_train.set_index(['user_id', 'product_id'])
new_features_train.head(n=10)

In [ ]:
#Remove features which are not predictors
new_features_train = new_features_train.drop(['eval_set', 'order_id','product_name'], axis=1)
new_features_train.head(n=10)

##3.2 Create Test DataFrame

In [ ]:
new_features_test = reduce_mem_usage(new_features[new_features.eval_set == 'test'])
new_features_test.head()

In [ ]:
#Create a unique index for the dataframe using user_id and product_id
new_features_test = new_features_test.set_index(['user_id', 'product_id'])
new_features.head()

In [ ]:
#Remove features which are not predictors
new_features_test = new_features_test.drop(['eval_set', 'order_id','product_name'], axis=1)
new_features.head()

In [ ]:
#Export file into CSV since keep crashing 
new_features_test.to_csv("new_features_test.csv",sep = ',')
files.download('new_features_test.csv')